# Quick Guide to *career_platform* Package

In [12]:
import uuid
uuid.uuid1().__str__()

'6b2deb99-b531-11ed-9b49-f894c274d899'

## 0. 导入



career_platform包的两种导入方式, 使用时任选其一即可:  
1. 将Career_Platform项目文件夹中的career_platform文件夹整体拷贝到你主程序的同级别文件夹下, 即可在主程序中`import career_platform as CP `导入使用. (不推荐)
2. 将Career_Platform项目文件夹所在的文件夹 (即它的父文件夹) 加入path, 以`from Career_Platform import career_platform as CP `导入使用, 示例:  
    ```python
    import os, sys
    sys.path.append("Career_Platform项目文件夹的父文件夹路径")
    from Career_Platform import career_platform as CP
    ```

In [1]:
# 这里用了第1种导入方式因为这个notebook刚好跟career_platform在同一文件夹下
import career_platform as CP
import datetime

[WARNING] failed to import dmPython module. ignore this message if you don't need dmPython


## 1. 各模块简介

### 1.1 Common: 通用数据类

`common`模块中包括了所有的通用数据类, 这些类是数据的载体, career_platform提供的所有算法都应使用这些数据类作为输入和输出

In [2]:
# Person类, 这里只初始化了部分Attribute, Person的全部Attribute见Person的注释
person1 = CP.common.Person(uuid="99A67C12-C5F0-3218-AD6A-C78BD80917B0", name="张老三", minzu="汉族", cur_adminrank="正处级", time_birth=datetime.date(1980, 12, 1))

# Experience类, 这里只初始化了部分Attribute, Experience的全部Attribute见Experience的注释
exp1 = CP.common.Experience(uuid="balabala", splitnum=0, person_uuid="99A67C12-C5F0-3218-AD6A-C78BD80917B0", 
                            text_raw="深圳市南山区桃源街道办科员、主任科员", time_start=datetime.date(1988, 12, 12), time_end=datetime.date(1992, 12, 1))
# 其他通用数据类见文档

### 1.2 Persistent: 持久化工具

`persistent.mapper`模块中包含了用于存取数据的许多Mapper。 一个Mapper只针对一种`common`中的数据类，以及一种特定的数据源。下面以`Experience`类的JSON文件Mapper类演示: 

In [3]:
jem = CP.persistent.mapper.JsonExperienceMapper()    # 获取一个JsonExperienceMapper实例，可以用它来读写JSON中存的Experience

exps_in_json = jem.getAll("./demo/sample_data/sample_data_exp.json")    # 用getAll函数读取sample_data_exp.json中的所有Experience

除了JSON文件的Mapper，数据库的Mapper肯定是必不可少的，下面演示用于在CAREER数据库存取`Experience`的Mapper类: 

运行下面的代码块前请先确保你的数据库是可用的！并且初始化成career_platform/persistent/CAREER.sql描述的样子。可以调用`CP.persistent.init_CareerDB()`函数初始化CAREER数据库和库表。

In [4]:
CP.persistent.init_CareerDB()   # 若想初始化数据库和库表, 可以运行这行代码.

cem = CP.persistent.mapper.CareerDBExperienceMapper()   # 获取用于数据库存取Experience的Mapper实例

# 我们可以先试试把刚刚从JSON文件中读出的所有Experience保存进数据库
cem.save(replace_by="exp_uuid", exp_list=exps_in_json)
# 如果运行成功，你现在可以去CAREER数据库的exp表里看看保存好的Experience记录


# 因为数据库比JSON文件牛逼很多, 所以数据库的Mapper支持更多的读取方式: 

# 按特定exp_uuid读取经历
exp_list = cem.getByExpuuid(uuid="7A01D8ED-016D-3D04-830E-884C79A54DAF")

# 按特定人的person_uuid读取经历
exp_list = cem.getByPersonuuid(person_uuid="D68FA2D4-ADC9-3B20-AC77-42585CD1D59F")

# 按经历在CAREER中的id读取经历 
# 【!!!请注意!!!】CAREER数据库中Experience的id是(exp_uuid, splitnum)组合主键而不是exp_uuid. 因为Experience经过兼职拆分后, 一个会变成多个, 因此exp_uuid并不唯一
exp = cem.getById(id=("7A01D8ED-016D-3D04-830E-884C79A54DAF", 0))

# getAll可以把数据库中全部的Experience读出来
exp_list = cem.getAll()

# 更多接口见文档或注释

execution success! CAREER_dev database initialized!


True

### 1.3 Algorithm: 核心算法  

`algorithm`模块目前包含四个子模块: `exp_parser`、`person_parser`、`label` 和 `network`，他们各自对外提供了一些函数, 且模块之间互不相关. 

In [5]:
# 这里只演示exp_parser中的三个函数: refine(), segment() 和 rebuild() , 他们具体的作用请看函数注释

exp1 = CP.common.Experience(uuid="88615C22-FB76-3A1D-8BE1-CAA2C369B045", 
                            text_raw="深圳广播电影电视集团编辑委员会委员、卫视频道总监、深圳市文广电文化传播有限公司总经理、深圳广电集团总裁助理", 
                            time_start = datetime.date(2012, 12, 1), 
                            time_end=datetime.date(2019, 3, 1), 
                            person_uuid="671430B6-8D28-3175-B87D-E81F2F5978C6")

exp1 = CP.algorithm.exp_parser.refine([exp1, ])[0]
print("\n================================== refine后的Experience: ======================================")
print(exp1)

exp1 = CP.algorithm.exp_parser.segment([exp1, ])[0]
print("\n================================== segment后的Experience: =====================================")
print(exp1)

exp_list = CP.algorithm.exp_parser.rebuild([exp1, ])
print("\n============================== rebuild后得到的多个Experience: ==================================")
for i in exp_list:
    print(i)


================================== refine后的Experience: ======================================
{'uuid': '88615C22-FB76-3A1D-8BE1-CAA2C369B045', 'splitnum': 0, 'ordernum': None, 'person_uuid': '671430B6-8D28-3175-B87D-E81F2F5978C6', 'text': None, 'text_token': None, 'text_raw': '深圳广播电影电视集团编辑委员会委员、卫视频道总监、深圳市文广电文化传播有限公司总经理、深圳广电集团总裁助理', 'text_rawpinyin': None, 'text_rawrefine': '深圳广播电影电视集团编辑委员会委员、卫视频道总监、深圳市文广电文化传播有限公司总经理、深圳广电集团总裁助理', 'text_rawsplit': '深圳广播电影电视集团编辑委员会委员|卫视频道总监|深圳市文广电文化传播有限公司总经理|深圳广电集团总裁助理', 'text_rawtoken': None, 'adminrank': None, 'time_start': datetime.date(2012, 12, 1), 'time_end': datetime.date(2019, 3, 1)}

================================== segment后的Experience: =====================================
{'uuid': '88615C22-FB76-3A1D-8BE1-CAA2C369B045', 'splitnum': 0, 'ordernum': None, 'person_uuid': '671430B6-8D28-3175-B87D-E81F2F5978C6', 'text': None, 'text_token': None, 'text_raw': '深圳广播电影电视集团编辑委员会委员、卫视频道总监、深圳市文广电文化传播有限公司总经理、深圳广电集团总裁助理', 'text_rawpinyin': None, 'text_rawr

## 2. 典型数据处理流程

### 2.1 经历数据解析流程

In [6]:
# 获取经历数据存取Mapper
jem = CP.persistent.mapper.JsonExperienceMapper()

# 读取经历数据
exp_list = jem.getAll("./demo/sample_data/sample_data_exp.json")

# 处理经历数据
exp_list = CP.algorithm.exp_parser.refine(exp_list, in_position=True, callback=CP.algorithm.tqdm_callback())
exp_list = CP.algorithm.exp_parser.segment(exp_list, in_position=True, callback=CP.algorithm.tqdm_callback())
exp_list = CP.algorithm.exp_parser.rebuild(exp_list, callback=CP.algorithm.tqdm_callback())

# 保存处理好的经历数据到数据库
cem = CP.persistent.mapper.CareerDBExperienceMapper()
cem.save(replace_by="exp_uuid", exp_list=exp_list)

[exp_parser]-rebuild: 100%|█████████▉| 2068/2069 [00:01<00:00, 2065.58it/s]


4732

### 2.2 经历、评价标签预测流程

In [7]:
# 经历标签==================================================

# 获取经历数据存取Mapper
cem = CP.persistent.mapper.CareerDBExperienceMapper()

# 读取经历数据
exp_list = cem.getAll()

# 从经历数据预测Label
label_list = CP.algorithm.label.label_exp(exp_list)

# 获取Label数据存取Mapper, 并保存预测出来的所有Label
clm = CP.persistent.mapper.CareerDBLabelMapper()
clm.save(replace_by="exp_uuid", label_list=label_list)

Bert Predict:  33%|███▎      | 7/21 [00:04<00:09,  1.54it/s]

In [3]:
# 评价标签==================================================

# 读取并保存评价
jevalm = CP.persistent.mapper.JsonEvaluationMapper()
eval_list = jevalm.getAll("./demo/sample_data/sample_data_evaluation.json")
cevalm = CP.persistent.mapper.CareerDBEvaluationMapper()
cevalm.save(replace_by="person_uuid", eval_list=eval_list)

# 从Evaluation预测Label
label_list = CP.algorithm.label.label_eval(eval_list=eval_list, callback=CP.algorithm.utils.tqdm_callback())

# 获取Label数据存取Mapper, 并保存预测出来的所有Label
clm = CP.persistent.mapper.CareerDBLabelMapper()
clm.save(replace_by="eval_uuid", label_list=label_list)

[labe_eval]-fuzz_match:  90%|█████████ | 9/10 [00:05<00:00,  1.51it/s]


366

### 2.3 人员数据处理流程

In [8]:
# 获取人员数据存取Mapper
jpm = CP.persistent.mapper.JsonPersonMapper()

# 读取人员数据
person_list_all = jpm.getAll("./demo/sample_data/sample_data_person.json")

# 处理人员数据
pass

# 保存人员数据
cpm = CP.persistent.mapper.CareerDBPersonMapper()
cpm.save(person_list_all)

180

### 2.4 组织机构树、关系网络生成流程

In [9]:
# 获取经历数据存取Mapper、读取经历数据
cem = CP.persistent.mapper.CareerDBExperienceMapper()
exp_list = cem.getAll()
# 获取人员数据存取Mapper、读取人员数据
cpm = CP.persistent.mapper.CareerDBPersonMapper()
person_list = cpm.getAll()

# 用经历和人员数据生成组织机构树、在neo4j中计算出关系网络等
octree = CP.algorithm.network.octree(exp_list=exp_list, person_list=person_list)

# 把neo4j和nid2resumes.json里的Relationship都导出
rel_list_neo4j = CP.algorithm.network.export_neo4j_relationship(callback=CP.algorithm.utils.tqdm_callback())
rel_list_nid2r = CP.algorithm.network.export_nid2resumes_relationship(callback=CP.algorithm.utils.tqdm_callback())

# 把导出的Relationship存进CAREER数据库
crm = CP.persistent.mapper.CareerDBRelationshipMapper()
crm.save(replace_by="person_uuid", relationship_list=rel_list_neo4j+rel_list_nid2r)

******** Building OCTree ****************************
******** Updating node interval *********************
******** Dumping nid2resumes dict *******************
******** Inserting 2654 YearUser Nodes **************
******** Inserting 2474 Trajectory Rels *************
******** Storing social net(col:114 sup_s:19 sup_c:8) 


[export Rels]-nid2resumes:  95%|█████████▍| 4783/5053 [00:00<00:00, 68823.24it/s]


636

In [10]:
# 读取人员数据
cpm = CP.persistent.mapper.CareerDBPersonMapper()
person_list_all = cpm.getAll()

# 匹配得到Relationship
rel_relshp_list = CP.algorithm.person_parser.export_rel_relationship()
alum_relshp_list = CP.algorithm.person_parser.export_alumni_relationship(person_list_all, callback=CP.algorithm.utils.tqdm_callback())
tx_relshp_list = CP.algorithm.person_parser.export_tx_relationship(person_list_all, callback=CP.algorithm.utils.tqdm_callback())

# 把匹配的Relationship存进CAREER数据库
crm = CP.persistent.mapper.CareerDBRelationshipMapper()
crm.save(replace_by="id", relationship_list=alum_relshp_list)
crm.save(replace_by="id", relationship_list=tx_relshp_list)

[person_parser]-export_tx_relshp:  18%|█▊        | 33/180 [00:00<00:00, 3004.12it/s]


22